https://www.quantlibguide.com/Instruments%20and%20pricing%20engines.html#other-pricing-methods

# Instruments and pricing engines
This notebook showcases a couple of features that the infrastructure of the library makes available; namely, it will show how instruments can use different so-called pricing engines to calculate their prices (each engine implementing a given model and/or numerical method) and how engines and instruments can be notified of changes in their input data and react accordingly.

# Setup

To begin, we import the QuantLib module and set up the global evaluation date.

In [1]:
import QuantLib as ql

today = ql.Date(7, ql.March, 2024)
ql.Settings.instance().evaluationDate = today

# The instrument

In this notebook, we’ll leave fixed-income and take a textbook instrument example: a European option.

Building the option requires only the specification of its contract, so its payoff (it’s a call option with strike at 100) and its exercise, three months from today’s date. The instrument doesn’t take any market data; they will be selected and passed later, depending on the calculation method.

In [3]:
option = ql.EuropeanOption(
    ql.PlainVanillaPayoff(ql.Option.Call, 100.0),
    ql.EuropeanExercise(ql.Date(7, ql.June, 2024)),
)

# A first pricing method

The different pricing methods are implemented as pricing engines holding the required market data. The first we’ll use is the one encapsulating the analytic Black-Scholes formula.

First, we collect the quoted market data. We’ll assume flat risk-free rate and volatility, so they can be expressed by SimpleQuote instances: they model numbers whose value can change and that can notify observers when this happens. The underlying value is at 100, the risk-free value at 1%, and the volatility at 20%.

In [5]:
u = ql.SimpleQuote(100.0)
r = ql.SimpleQuote(0.01)
σ = ql.SimpleQuote(0.20)

In order to build the engine, the market data are encapsulated in a Black-Scholes process object. The process can use full-fledged term structures, so it can include time-dependency and smiles. In this case, for simplicity, we build flat curves for the risk-free rate and the volatility.

In [7]:
riskFreeCurve = ql.FlatForward(
    0, ql.TARGET(), ql.QuoteHandle(r), ql.Actual360()
)
volatility = ql.BlackConstantVol(
    0, ql.TARGET(), ql.QuoteHandle(σ), ql.Actual360()
)

Now we can instantiate the process with the underlying value and the curves we just built. The inputs are all stored into handles, so that we could change the quotes and curves used if we wanted. I’ll skip over this for the time being.

In [11]:
process = ql.BlackScholesProcess(
    ql.QuoteHandle(u),
    ql.YieldTermStructureHandle(riskFreeCurve),
    ql.BlackVolTermStructureHandle(volatility),
)

Once we have the process, we can finally use it to build the engine…

In [13]:
engine = ql.AnalyticEuropeanEngine(process)

…and once we have the engine, we can set it to the option and evaluate the latter.

In [15]:
option.setPricingEngine(engine)

In [17]:
print(option.NPV())

4.155543462156206


Depending on the instrument and the engine, we can also ask for other results; in this case, we can ask for Greeks.

In [19]:
print(option.delta())
print(option.gamma())
print(option.vega())

0.5302223303784392
0.03934493301271913
20.109632428723106


# Market changes

As I mentioned, market data are stored in Quote instances and thus can notify the option when any of them changes. We don’t have to do anything explicitly to tell the option to recalculate: once we set a new value to the underlying, we can simply ask the option for its NPV again and we’ll get the updated value.

In [21]:
u.setValue(105.0)
print(option.NPV())

7.27556357927846


Other market data also affect the value, of course.

In [23]:
r.setValue(0.02)
print(option.NPV())

7.448878025811257


In [25]:
σ.setValue(0.15)
print(option.NPV())

6.596556078273312


# Date changes

Just as it does when inputs are modified, the value also changes if we advance the evaluation date. Let’s look first at the value of the option when its underlying is worth 105 and there’s still three months to exercise…

In [27]:
u.setValue(105.0)
r.setValue(0.01)
σ.setValue(0.20)
print(option.NPV())

7.27556357927846


…and then move to a date two months before exercise.

In [29]:
ql.Settings.instance().evaluationDate = ql.Date(7, ql.April, 2024)

Again, we don’t have to do anything explicitly: we just ask the option for its value, and we see that it has decreased as expected.

In [31]:
print(option.NPV())

6.535204576446796


# A note on the option value on its exercise date
In the default library configuration, the instrument is considered to have expired when it reaches the exercise date, so its returned value goes down to 0.

In [33]:
ql.Settings.instance().evaluationDate = ql.Date(7, ql.June, 2024)

In [35]:
print(option.NPV())

0.0


It’s possible to tweak the configuration so that the instrument is still considered alive.

In [37]:
ql.Settings.instance().includeReferenceDateEvents = True

The above changes the settings, but doesn’t send a notification to the instrument so we need to trigger an explicit recalculation. Normally, though, one would change the setting at the start of one’s program so this step would be unnecessary.

In [39]:
option.recalculate()

print(option.NPV())

5.0


However, this is not guaranteed to work for all pricing engines, since each one must manage this case specifically; and even when they return a price, they are not guaranteed to return meaningful values for all available results. For instance, at the time of this writing, the cell below will print two NaNs; if it doesn’t, please send me a line so I can update this text.

In [41]:
print(option.delta())
print(option.vega())

nan
nan


# Other pricing methods

As I mentioned, the instrument machinery allows us to use different pricing methods. For comparison, I’ll first set the input data back to what they were previously and output the Black-Scholes price.

In [43]:
ql.Settings.instance().evaluationDate = today
u.setValue(105.0)
r.setValue(0.01)
σ.setValue(0.20)

In [45]:
print(option.NPV())

7.27556357927846


Let’s say that we want to use a Heston model to price the option. What we have to do is to instantiate the corresponding class with the desired inputs (here I’ll skip the calibration and pass precalculated parameters)…

In [47]:
model = ql.HestonModel(
    ql.HestonProcess(
        ql.YieldTermStructureHandle(riskFreeCurve),
        ql.YieldTermStructureHandle(
            ql.FlatForward(0, ql.TARGET(), 0.0, ql.Actual360())
        ),
        ql.QuoteHandle(u),
        0.04,
        0.1,
        0.01,
        0.05,
        -0.75,
    )
)

…pass it to the corresponding engine, and set the new engine to the option.

In [49]:
engine = ql.AnalyticHestonEngine(model)
option.setPricingEngine(engine)

Asking the option for its NPV will now return the value according to the new model.

In [51]:
print(option.NPV())

7.295356086978635


# Lazy recalculation

One last thing. Up to now, we haven’t really seen evidence of notifications going around. After all, the instrument might just have recalculated its value every time we asked it, regardless of notifications. What I’m going to show, instead, is that the option doesn’t just recalculate every time anything changes; it also avoids recalculations when nothing has changed.

We’ll switch to a Monte Carlo engine, which takes a few seconds to run the required simulation.

In [53]:
engine = ql.MCEuropeanEngine(
    process, "PseudoRandom", timeSteps=20, requiredSamples=500_000
)
option.setPricingEngine(engine)

When we ask for the option value, we have to wait a noticeable time for the calculation to finish (for those of you reading this in a non-interactive way, I’ll also have the notebook output the time)…

In [55]:
%time print(option.NPV())

7.265651518182009
CPU times: user 2.36 s, sys: 45.1 ms, total: 2.41 s
Wall time: 2.42 s


…but a second call to the NPV method will be instantaneous when made before anything changes. In this case, the option didn’t calculate its value; it just returned the result that it cached from the previous call.

In [57]:
%time print(option.NPV())

7.265651518182009
CPU times: user 585 μs, sys: 476 μs, total: 1.06 ms
Wall time: 693 μs


If we change anything (e.g., the underlying value)…

In [59]:
u.setValue(104.0)

…the option is notified of the change, and the next call to NPV will again take a while.

In [61]:
option

<QuantLib.QuantLib.EuropeanOption; proxy of <Swig Object of type 'ext::shared_ptr< EuropeanOption > *' at 0x10d1bd3e0> >

In [63]:
option.delta

<bound method OneAssetOption.delta of <QuantLib.QuantLib.EuropeanOption; proxy of <Swig Object of type 'ext::shared_ptr< EuropeanOption > *' at 0x10d1bd3e0> >>